In [14]:
from pathlib import Path
from omnibelt import toposort, load_json, save_json, pformat, pformat_vars
from itertools import product, combinations
# import pyperclip
import random
from PIL import Image
from graphviz import Digraph
import matplotlib.pyplot as plt
import io
# import clipboard
import networkx as nx
from tqdm.notebook import tqdm
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from omniply import tool, ToolKit, Context

import numpy as np
import torch
from causalbenchmark.novo.simulation.models import Bernoulli, ConditionalBernoulli, BernoulliNetwork
from causalbenchmark.novo.seeding import NewsHeadlines, StatisticsPrompting, Story
# from causalbenchmark.novo.seeding.prompting import GraphInfo
from causalbenchmark.novo.templating import FixedTemplate, SimpleTemplater
from causalbenchmark.novo import misc

In [5]:
story_names = [
	'test19',
	'test20',
	'test21',
]
stories = []
for name in story_names:
	story = Story(story_root=misc.data_root() / 'stories', story_id=name).populate_defaults()
	stories.append(story)
	
	story['stats'] = {int(k): v for k, v in story['stats'].items()}
	story['verbs'] = {k: {int(k2): v2 for k2, v2 in v.items()} for k, v in story['verbs'].items()}
	for k, v in story['verbs'].items():
		for k2, v2 in v.items():
			if 'value' in v2:
				v2['value'] = {int(k3): v3 for k3, v3 in v2['value'].items()}
	story['questions']['questions'] = {int(k): v for k, v in story['questions']['questions'].items()}
len(stories)

In [8]:
ctx = random.choice(stories)

In [9]:
rawverbs = ctx['verbs']
rawvars = {node['name']: node for node in ctx['nodes']}

In [10]:
next(iter(rawverbs.values()))

{1: {'variable': ['the local economic strength',
   'the robustness of the local economy']},
 2: {'subject': 'the area',
  'value': {0: ['experiences a struggling economy',
    'is facing economic difficulties'],
   1: ['enjoys a thriving economy', 'benefits from a strong economy']}},
 3: {'value': {0: ['the economy is struggling',
    'economic challenges are prevalent'],
   1: ['the economy is robust', "there's economic prosperity"]}},
 4: {'domain': 'regions',
  'value': {0: ['are economically struggling', 'face economic challenges'],
   1: ['are economically strong', 'exhibit economic prosperity']}},
 5: {'value': {0: ['the economy falters', 'economic downturns are seen'],
   1: ['the economy flourishes', 'economic growth is evident']}},
 6: {'value': {0: ['In regions with a weak economy',
    'Where the economy is struggling'],
   1: ['In areas with a strong economy', 'Where the economy is flourishing']}},
 7: {'value': {0: ['if the local economy weakens',
    'if economic conditi

In [13]:
def story_atoms(story):
	varverbs = {}
	for var, raw in story['verbs'].items():
		info = {'values': {0: {}, 1: {}}}
		
		info['variable'] = raw[1]['variable']
		info['subject'] = raw[2]['subject']
		info['domain'] = raw[4]['domain']
		
		for i in [0, 1]:
			info_val = info['values'][i]
			info_val['verb'] = raw[2]['value'][i] # goes with subject
			info_val['phrase'] = raw[3]['value'][i]
			info_val['descriptor'] = raw[4]['value'][i] # goes with domain
			info_val['event'] = raw[5]['value'][i]
			info_val['conditional'] = raw[6]['value'][i]
			info_val['interventional'] = raw[7]['value'][i]
			
		varverbs[var] = info
	
	story['sheet'] = varverbs
	return story

for story in stories:
	story_atoms(story)

In [ ]:
precise_templates = [
	['there', 'is', 'a', '{quantity}', '{prob_word}', 'that', '{position}'],
	['the', '{chance_word}', 'that', '{position}', '{"are" if chance_word == "odds" else "is"}', '{quantity}'],
	['{quantity}', 'of', 'the', 'time', '{position}'],
]

position_templates = [
	['{subject}', '{verb}',],
	['{domain}', '{descriptor}'],
	['{phrase}',],
	['{event}',],
]

prob_words = ['probability', 'chance', 'likelihood']
chance_words = ['probability', 'chance', 'likelihood', 'odds']



In [18]:
def detok(tokens):
	toks = [', ' if tok == ',' else tok for tok in tokens]
	sentence = ' '.join(toks) + '.'
	return sentence.capitalize()

In [ ]:
# 1. One of the variables is {variable}.
# 2. We estimate {subject} [often/usually/sometimes/rarely/etc.] {value}.
# 3. There is a [number]% chance that {value}.
# 4. [number]% of {domain} {value}.
# 5. [number]% of the time {value}.
# 6. Conditional sentence: {value}, [some consequence].
# 7. Interventional: If {value}, [some effect].

In [ ]:
class Decision(AbstractMultiGadget):
	def __init__(self, name: str, choices: list['AbstractGadget'] | dict[str, 'AbstractGadget'], id_name: str = None, **kwargs):
		if id_name is None:
			id_name = f'{name}_id'
		if isinstance(choices, list):
			choices = {i: c for i, c in enumerate(choices)}
		super().__init__(**kwargs)
		self._choice_ids = list(choices.keys())
		self._choices = choices
		self._name = name
		self._id_name = id_name
		
	def __len__(self):
		return len(self._choice_ids)
		
	def gizmos(self):
		yield from (self._id_name, self._name)
	
	def pick(self, ctx: Context = None):
		return random.choice(list(self._choices.keys())) if ctx is None else ctx.select(list(self._choice_ids))
	
	def grab_from(self, ctx: Context, gizmo: str = None):
		if gizmo is None:
			gizmo = self._name
		if gizmo == self._id_name:
			pass
			
			
			return self._choices[ctx[self._id_name]]
		
		self._name = ctx[self._name]
		self._id_name = ctx[self._id_name]
		return self
	
	pass


class ClauseVerbalizer(Context):
	def __init__(self, rng=None, **kwargs):
		if rng is None:
			rng = misc.get_rng(rng)
		super().__init__(**kwargs)
		self._rng = rng
			
			
	def populate_default(self):
		
		
		
		return self
	
	